In [1]:
!date

Sun Feb 27 18:07:25 PST 2022


This notebook uses [Stan](https://mc-stan.org) and [Bean Machine](https://beanmachine.org) to fit a linear model with outliers. The model and data are taken from Jake VanderPlas's post [Frequentism and Bayesianism II: When Results Differ](http://jakevdp.github.io/blog/2014/06/06/frequentism-and-bayesianism-2-when-results-differ/). Specifically, this is [Example #2](http://jakevdp.github.io/blog/2014/06/06/frequentism-and-bayesianism-2-when-results-differ/#Example-#2:-Linear-Fit-with-Outliers) from his post. There's also a very nice discussion of Bayesian modeling for handling outliers in §8.3 of [Data Analysis - A Bayesian Tutorial](https://www.goodreads.com/book/show/1430545.Data_Analysis) by D.S. Sivia. Finally, this discussion is complementary to the official Bean Machine [Robust Linear Regression](https://beanmachine.org/docs/tutorials/#robust-linear-regression) tutorial.

Points are outliers with probability $\theta$. All observations are drawn from a normal distribution. For points that are not outliers, we use the estimated observation error $e_i$ as the standard deviation. For outliers, we use a constant standard deviation $\sigma_o$, which is estimated as part of the model. The model description below, $z$ is a latent indicator for whether or not the point is an outlier.

$$
\begin{align}
\theta & \sim \mathrm{Beta}(2, 5) \\
\sigma_o & \sim \mathrm{Gamma}(2, 2) \\
\beta_0 & \sim \mathcal{N}(0, 10) \\
\beta_1 & \sim \mathcal{N}(0, 10) \\
z & \sim \mathrm{Bernouilli}(\theta) \\
y & \sim \mathcal{N}(\beta_0 + \beta_1 x, \sigma_{z[i]}) \\
\end{align}
$$

where

$$
\sigma_{z[i]} =
    \begin{cases}
    e_i, & z_i = 0 \\
    \sigma_o, & z_i = 1 \\
    \end{cases}
$$

For both the Bean Machine and Stan implementations, we marginalize out the latent discrete variable $z$, as described [here](https://mc-stan.org/docs/2_29/stan-users-guide/summing-out-the-responsibility-parameter.html) in the Stan documentation.

$$
p(y_i | \theta, \beta_0, \beta_1, \sigma_o) = \theta \mathcal{N}(\beta_0 + \beta_1 x_i, \sigma_o) + (1 - \theta) \mathcal{N}(\beta_0 + \beta_1 x_i, e_i)
$$

In [43]:
# This is needed for PyStan to run in a Jupyter notebook
import nest_asyncio
nest_asyncio.apply()

In [2]:
import altair
import beanmachine.ppl as bm
import numpy as np
import pandas as pd
import torch
import torch.distributions as dist

In [29]:
BLUE = "#4e79a7"
ORANGE = "#f28e2b"
RED = "#e15759"
CYAN = "#76b7b2"
GREEN = "#59a14f"
YELLOW = "#edc948"
PURPLE = "#b07aa1"
PINK = "#ff9da7"
BROWN = "#9c755f"
GRAY = "#bab0ac"

In [3]:
x = torch.tensor(
    [0, 3, 9, 14, 15, 19, 20, 21, 30, 35, 40, 41, 42, 43, 54, 56, 67, 69, 72, 88],
    dtype=torch.float32,
)
y = torch.tensor(
    [33, 68, 34, 34, 37, 71, 37, 44, 48, 49, 53, 49, 50, 48, 56, 60, 61, 63, 44, 71],
    dtype=torch.float32,
)
err = torch.tensor(
    [
        3.6,
        3.9,
        2.6,
        3.4,
        3.8,
        3.8,
        2.2,
        2.1,
        2.3,
        3.8,
        2.2,
        2.8,
        3.9,
        3.1,
        3.4,
        2.6,
        3.4,
        3.7,
        2.0,
        3.5,
    ]
)


In [31]:
error_bars = (
    altair.Chart()
    .mark_errorbar(color=BLUE)
    .encode(altair.X("x"), altair.Y("y"), altair.YError("err"))
)
points = (
    altair.Chart()
    .mark_circle(color=BLUE)
    .encode(x=altair.X("x"), y=altair.Y("y", scale=altair.Scale(zero=False)))
)
altair.layer(points, error_bars, data=pd.DataFrame({"x": x, "y": y, "err": err}))


alt.LayerChart(...)

In [52]:
STAN_CODE = """
data {
    int<lower=0> n_obs;
    vector[n_obs] x;
    vector[n_obs] y;
    vector[n_obs] err;
}

parameters {
    real beta0;
    real beta1;
    real<lower=0> sigma_o;
    real<lower=0, upper=1> theta;
}

model {
    real mu;
    beta0 ~ normal(0, 10);
    beta1 ~ normal(0, 10);
    sigma_o ~ gamma(2, 2);
    theta ~ beta(2, 5);

    for (i in 1:n_obs) {
        mu = beta0 + beta1 * x[i];
        target += log_mix(theta, normal_lpdf(y[i] | mu, sigma_o ), normal_lpdf(y[i] | mu, err[i] ));
    }
}
"""

In [53]:
posterior = stan.build(
    program_code=STAN_CODE,
    data={"n_obs": len(x), "x": x.tolist(), "y": y.tolist(), "err": err.tolist()},
    random_seed=332211,
)


Building...

In file included from /Users/tsmall/Library/Caches/httpstan/4.7.0/models/b3zskmbi/model_b3zskmbi.cpp:2:
In file included from /Users/tsmall/src/python-bean-machine/lib/python3.9/site-packages/httpstan/include/stan/model/model_header.hpp:4:
In file included from /Users/tsmall/src/python-bean-machine/lib/python3.9/site-packages/httpstan/include/stan/math.hpp:19:
In file included from /Users/tsmall/src/python-bean-machine/lib/python3.9/site-packages/httpstan/include/stan/math/rev.hpp:8:
In file included from /Users/tsmall/src/python-bean-machine/lib/python3.9/site-packages/httpstan/include/stan/math/rev/core.hpp:28:
In file included from /Users/tsmall/src/python-bean-machine/lib/python3.9/site-packages/httpstan/include/stan/math/rev/core/operator_addition.hpp:6:
/Users/tsmall/src/python-bean-machine/lib/python3.9/site-packages/httpstan/include/stan/math/prim/err/check_matching_dims.hpp:33:8: warning: unused variable 'error' [-Wunused-variable]
  bool error = false;
       ^
/Users/tsmall/

ld: warning: direct access in function 'long double boost::math::detail::bessel_j0<long double>(long double)' from file '/Users/tsmall/src/python-bean-machine/lib/python3.9/site-packages/httpstan/stan_services.o' to global weak symbol 'long double boost::math::detail::bessel_j0<long double>(long double)::Q2' from file 'build/temp.macosx-12-x86_64-3.9/Users/tsmall/Library/Caches/httpstan/4.7.0/models/b3zskmbi/model_b3zskmbi.o' means the weak symbol cannot be overridden at runtime. This was likely caused by different translation units being compiled with different visibility settings.
ld: warning: direct access in function 'long double boost::math::detail::bessel_j0<long double>(long double)' from file '/Users/tsmall/src/python-bean-machine/lib/python3.9/site-packages/httpstan/stan_services.o' to global weak symbol 'long double boost::math::detail::bessel_j0<long double>(long double)::P2' from file 'build/temp.macosx-12-x86_64-3.9/Users/tsmall/Library/Caches/httpstan/4.7.0/models/b3zskmb

In [54]:
fit = posterior.sample(num_chains=4, num_samples=1000)

Sampling:   0%
Sampling:  25% (2000/8000)
Sampling:  50% (4000/8000)
Sampling:  75% (6000/8000)
Sampling: 100% (8000/8000)
Sampling: 100% (8000/8000), done.
Messages received during sampling:
  Gradient evaluation took 2.2e-05 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.22 seconds.
  Adjust your expectations accordingly!
  Gradient evaluation took 3.7e-05 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.37 seconds.
  Adjust your expectations accordingly!
  Gradient evaluation took 4.3e-05 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.43 seconds.
  Adjust your expectations accordingly!
  Gradient evaluation took 4.4e-05 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.44 seconds.
  Adjust your expectations accordingly!


In [55]:
df_summary = fit.to_frame()

In [56]:
df_summary.describe(percentiles=[0.05, 0.5, 0.95]).T

,count,mean,std,min,5%,50%,95%,max
parameters,,,,,,,,
lp__,4000.0,-96.847199,1.506942,-107.462114,-99.743475,-96.512814,-95.069789,-94.743005
accept_stat__,4000.0,0.929085,0.098290,0.136147,0.730945,0.966989,1.000000,1.000000
stepsize__,4000.0,0.337962,0.011483,0.320605,0.320605,0.340878,0.349488,0.349488
treedepth__,4000.0,3.104250,0.693904,1.000000,2.000000,3.000000,4.000000,4.000000
n_leapfrog__,4000.0,10.642000,4.507873,1.000000,3.000000,11.000000,15.000000,31.000000
divergent__,4000.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
energy__,4000.0,98.864675,2.049025,94.910001,96.098780,98.561324,102.544776,112.184034
beta0,4000.0,32.198265,1.863560,25.534571,29.322431,32.081496,35.431268,42.271615
beta1,4000.0,0.447492,0.041893,0.267811,0.376613,0.448666,0.514474,0.606398


In [5]:
@bm.random_variable
def beta_0() -> bm.RVIdentifier:
    return dist.Normal(0, 10)


@bm.random_variable
def beta_1() -> bm.RVIdentifier:
    return dist.Normal(0, 10)


@bm.random_variable
def sigma_o() -> bm.RVIdentifier:
    return dist.Gamma(1, 1)


@bm.random_variable
def theta() -> bm.RVIdentifier:
    return dist.Beta(2, 5)


@bm.functional
def likelihood(i: int) -> float:
    mu = beta_0() + beta_1() * x[i]
    ll1 = torch.log(theta()) + dist.Normal(mu, sigma_o()).log_prob(y[i])
    ll2 = torch.log(1 - theta()) + dist.Normal(mu, err[i]).log_prob(y[i])
    return torch.logaddexp(ll1, ll2).exp()


@bm.random_variable
def d(i: int) -> bm.RVIdentifier:
    return dist.Bernoulli(likelihood(i))


In [6]:
altair.Chart(
    pd.DataFrame().assign(
        theta=torch.arange(0, 1, 0.01),
        p=lambda df: dist.Beta(2, 5).log_prob(torch.tensor(df["theta"])).exp(),
    )
).mark_line().encode(x="theta", y="p")


alt.Chart(...)

In [7]:
queries = [beta_0(), beta_1(), sigma_o(), theta()]
observations = {d(i): torch.tensor(1.0) for i in range(len(x))}

In [8]:
samples = bm.GlobalNoUTurnSampler().infer(
    queries=queries,
    observations=observations,
    num_samples=4000,
    num_adaptive_samples=4000,
    num_chains=1,
)

Samples collected: 100%|██████████| 8000/8000 [53:42<00:00,  2.48it/s]     


In [9]:
bm.Diagnostics(samples).summary()

,avg,std,2.5%,50%,97.5%,n_eff
beta_0()[],31.532238,1.700976,28.297164,31.525530,34.900478,1761.174072
beta_1()[],0.462220,0.039588,0.382067,0.462829,0.538872,1697.758179
sigma_o()[],12.869358,2.022978,9.426688,12.718417,17.258504,2320.917480
theta()[],0.270068,0.106900,0.095128,0.260704,0.509922,2611.940674


In [33]:
mu = 31.5 + 0.5 * x
z1 = dist.Normal(mu, err).log_prob(y).exp() * 0.73
z2 = dist.Normal(mu, 12.9).log_prob(y).exp() * 0.27

In [34]:
p_outlier = z2 / (z1 + z2)

In [35]:
p_outlier

tensor([0.1006, 1.0000, 0.0901, 0.1805, 0.1100, 1.0000, 0.3247, 0.0856, 0.0749,
        0.0982, 0.0733, 0.1218, 0.1188, 0.2324, 0.1114, 0.0705, 0.1566, 0.1254,
        1.0000, 0.1775])

In [36]:
error_bars = (
    altair.Chart()
    .mark_errorbar()
    .encode(
        altair.X("x"), altair.Y("y"), altair.YError("err"), altair.Color("is_outlier")
    )
)
points = (
    altair.Chart()
    .mark_circle()
    .encode(
        altair.X("x"),
        altair.Y("y", scale=altair.Scale(zero=False)),
        altair.Color("is_outlier"),
    )
)
altair.layer(
    points,
    error_bars,
    data=pd.DataFrame({"x": x, "y": y, "err": err, "is_outlier": p_outlier > 0.5}),
)


alt.LayerChart(...)